# Automated ML

TODO: Import Dependencies. In the cell below, import all the dependencies that you will need to complete the project.

In [59]:
import numpy as np
import pandas as pd
import os
import glob 
import os

from datetime import datetime
from Train_Model import clean_data
from sklearn.model_selection import train_test_split

## Dataset

### Overview
TODO: In this markdown cell, give an overview of the dataset you are using. Also mention the task you will be performing.

In the current 


TODO: Get data. In the cell below, write code to access the data you will be using in this project. Remember that the dataset needs to be external.

In [60]:
data = pd.read_csv('payer_level_features.csv')

In [61]:
x, y = clean_data(data)
y_dataframe = pd.DataFrame(y)

In [62]:
all_data = pd.concat([x,y_dataframe], axis = 1)

In [63]:
Train, Test = train_test_split(
       all_data , test_size=0.33, random_state=0)

Y_Train = Train["FLAG_BAD_DEBT"]
Y_Test = Test["FLAG_BAD_DEBT"]

In [64]:
X_Train = Train.drop(['Payer', 'FLAG_BAD_DEBT', 'Bankruptcy month'], axis=1)
X_Test = Test.drop(['Payer', 'FLAG_BAD_DEBT', 'Bankruptcy month'], axis=1)

In [65]:
print('Shape X_train: '+str(X_Train.shape))
print('Shape X_test: '+str(X_Test.shape))
print('Shape y_train: '+str(Y_Train.shape))
print('Shape y_test: '+str(Y_Test.shape))

Shape X_train: (4072, 71)
Shape X_test: (2006, 71)
Shape y_train: (4072,)
Shape y_test: (2006,)


In [66]:
print ("# Bad Debts in training data",Y_Train.sum())
print ("# Bad Debts in test data",Y_Test.sum())

# Bad Debts in training data 28
# Bad Debts in test data 15


In [95]:
Y_Train = Y_Train.rename('y')

In [88]:
from azureml.core import Dataset

X_Train.to_csv("Data/train_data.csv", index = False)

data_store = ws.get_default_datastore()

data_store.upload(src_dir = "./Data", target_path = 'project_3', overwrite = True,  show_progress = True)
train_data = Dataset.Tabular.from_delimited_files(path=data_store.path("project_3/train_data.csv"))

Uploading an estimated of 1 files
Uploading ./Data/train_data.csv
Uploaded ./Data/train_data.csv, 1 files out of an estimated total of 1
Uploaded 1 files


In [67]:
import azureml.core
from azureml.core.experiment import Experiment
from azureml.core.workspace import Workspace

## Login to Azure 

In [70]:
from azureml.core import Workspace, Experiment

ws = Workspace.from_config()

# choose a name for experiment
experiment_name = 'auto-ml-bankrupt'

experiment=Experiment(ws, experiment_name)

print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')

run = experiment.start_logging()

Workspace name: ops_analytics-workspace
Azure region: westeurope
Subscription id: 73f88e6b-3a35-4612-b550-555157e7059f
Resource group: ops_analytics-rg-gb-dev


## Choose the Compute to Calculate 

In [76]:
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException 

# Choose a name for your CPU cluster
amlcompute_cluster_name = "Demo-Cluster"

# Verify that cluster does not exist already
try:
    aml_compute = ComputeTarget(workspace=ws, name=amlcompute_cluster_name)
    print('Found existing cluster, use it.')
except ComputeTargetException:
    print("Creating a new compute cluster...")
    compute_config =  AmlCompute.provisioning_configuration(vm_size='STANDARD_D2S_V3', max_nodes=4, min_nodes=0, vm_priority='lowpriority') 
    aml_compute = ComputeTarget.create(ws, amlcompute_cluster_name, compute_config)

# Can poll for a minimum number of nodes and for a specific timeout. 
# If no min node count is provided it uses the scale settings for the cluster.
    
#aml_compute.wait_for_completion(show_output=True)
aml_compute.wait_for_completion(show_output=True, min_node_count = None, timeout_in_minutes = 60)
#compute_target.wait_for_completion(show_output=True, min_node_count = None, timeout_in_minutes = 60)


# use get_status() to get a detailed status for the current cluster. 
print(aml_compute.get_status().serialize())

Found existing cluster, use it.
Succeeded
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned
{'currentNodeCount': 0, 'targetNodeCount': 0, 'nodeStateCounts': {'preparingNodeCount': 0, 'runningNodeCount': 0, 'idleNodeCount': 0, 'unusableNodeCount': 0, 'leavingNodeCount': 0, 'preemptedNodeCount': 0}, 'allocationState': 'Steady', 'allocationStateTransitionTime': '2021-10-06T17:13:51.297000+00:00', 'errors': None, 'creationTime': '2021-09-17T05:18:08.686648+00:00', 'modifiedTime': '2021-09-17T05:18:24.335748+00:00', 'provisioningState': 'Succeeded', 'provisioningStateTransitionTime': None, 'scaleSettings': {'minNodeCount': 0, 'maxNodeCount': 1, 'nodeIdleTimeBeforeScaleDown': 'PT1800S'}, 'vmPriority': 'Dedicated', 'vmSize': 'STANDARD_D2S_V3'}


## AutoML Configuration

TODO: Explain why you chose the automl settings and cofiguration you used below.

In [101]:
from azureml.train.automl import AutoMLConfig

# TODO: Put your automl settings here

automl_settings = {
    "experiment_timeout_minutes": 40,
    "max_concurrent_iterations": 5,
    "primary_metric" : 'AUC_weighted'
}

# TODO: Put your automl config here
automl_config = AutoMLConfig(compute_target=aml_compute,
                             task = "classification",
                             training_data=train_data,
                             label_column_name="y",   
                             enable_early_stopping= True,
                             featurization= 'auto',
                             debug_log = "automl_errors.log",
                             **automl_settings
                            )

In [102]:
# Submit your automl run
automl_run = experiment.submit(automl_config,show_output = True)
RunDetails(automl_run).show()
automl_run.wait_for_completion()

Submitting remote run.
No run_configuration provided, running on Demo-Cluster with default configuration
Running on remote compute: Demo-Cluster
The validation results are as follows:
Y is empty


ValidationException: ValidationException:
	Message: Failed to execute the requested operation: data/settings validation. Error details: Validation error(s): [{
    "additional_properties": {
        "debugInfo": null
    },
    "code": "UserError",
    "severity": 2,
    "message": "Y is empty",
    "message_format": "{dataName} is empty",
    "message_parameters": {
        "dataName": "Y"
    },
    "reference_code": null,
    "details_uri": null,
    "target": "training_data",
    "details": [
        {
            "additional_properties": {
                "debugInfo": null
            },
            "code": null,
            "severity": null,
            "message": "null",
            "message_format": null,
            "message_parameters": {},
            "reference_code": null,
            "details_uri": null,
            "target": null,
            "details": [],
            "inner_error": null,
            "additional_info": null
        }
    ],
    "inner_error": {
        

In [ ]:
# TODO: Submit your experiment
remote_run = experiment.submit(automl_config)

## Run Details

OPTIONAL: Write about the different models trained and their performance. Why do you think some models did better than others?

TODO: In the cell below, use the `RunDetails` widget to show the different experiments.

## Best Model

TODO: In the cell below, get the best model from the automl experiments and display all the properties of the model.



In [ ]:
#TODO: Save the best model

## Model Deployment

Remember you have to deploy only one of the two models you trained but you still need to register both the models. Perform the steps in the rest of this notebook only if you wish to deploy this model.

TODO: In the cell below, register the model, create an inference config and deploy the model as a web service.

TODO: In the cell below, send a request to the web service you deployed to test it.

TODO: In the cell below, print the logs of the web service and delete the service

**Submission Checklist**
- I have registered the model.
- I have deployed the model with the best accuracy as a webservice.
- I have tested the webservice by sending a request to the model endpoint.
- I have deleted the webservice and shutdown all the computes that I have used.
- I have taken a screenshot showing the model endpoint as active.
- The project includes a file containing the environment details.
